In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
dataframe1 = pd.read_csv("/home/raj/Downloads/2017-fordgobike-tripdata.csv.csv", sep=',', header=0, low_memory=False , encoding='latin-1')
dataframe2 = pd.read_csv("/home/raj/Downloads/201801-fordgobike-tripdata.csv", sep=',', header=0, low_memory=False , encoding='latin-1')
dataframe3 = pd.read_csv("/home/raj/Downloads/201802-fordgobike-tripdata.csv", sep=',', header=0, low_memory=False , encoding='latin-1')
dataframe4 = pd.read_csv("/home/raj/Downloads/201803-fordgobike-tripdata.csv", sep=',', header=0, low_memory=False , encoding='latin-1')

In [3]:
dataframe = pd.concat([dataframe1, dataframe2, dataframe3, dataframe4], join='inner', ignore_index=True)
dataframe.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender
0,80110,2017-12-31 16:57:39.6540,2018-01-01 15:12:50.2450,74,Laguna St at Hayes St,37.776435,-122.426244,43,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96,Customer,1987.0,Male
1,78800,2017-12-31 15:56:34.8420,2018-01-01 13:49:55.6170,284,Yerba Buena Center for the Arts (Howard St at ...,37.784872,-122.400876,96,Dolores St at 15th St,37.766210,-122.426614,88,Customer,1965.0,Female
2,45768,2017-12-31 22:45:48.4110,2018-01-01 11:28:36.8830,245,Downtown Berkeley BART,37.870348,-122.267764,245,Downtown Berkeley BART,37.870348,-122.267764,1094,Customer,NaN,NaN
3,62172,2017-12-31 17:31:10.6360,2018-01-01 10:47:23.5310,60,8th St at Ringold St,37.774520,-122.409449,5,Powell St BART Station (Market St at 5th St),37.783899,-122.408445,2831,Customer,NaN,NaN
4,43603,2017-12-31 14:23:14.0010,2018-01-01 02:29:57.5710,239,Bancroft Way at Telegraph Ave,37.868813,-122.258764,247,Fulton St at Bancroft Way,37.867789,-122.265896,3167,Subscriber,1997.0,Female


In [4]:
dataframe.info() # there is no null data in the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 832602 entries, 0 to 832601
Data columns (total 15 columns):
duration_sec               832602 non-null int64
start_time                 832602 non-null object
end_time                   832602 non-null object
start_station_id           832602 non-null int64
start_station_name         832602 non-null object
start_station_latitude     832602 non-null float64
start_station_longitude    832602 non-null float64
end_station_id             832602 non-null int64
end_station_name           832602 non-null object
end_station_latitude       832602 non-null float64
end_station_longitude      832602 non-null float64
bike_id                    832602 non-null int64
user_type                  832602 non-null object
member_birth_year          741095 non-null float64
member_gender              741286 non-null object
dtypes: float64(5), int64(4), object(6)
memory usage: 95.3+ MB


## Average Trip duration in seconds

In [5]:
dataframe['duration_sec'].mean()

1014.2486866474018

## In Minutes Divide the above result with 60 

In [6]:
dataframe['duration_sec'].mean()/60

16.904144777456697

## Standard Deviation in seconds

In [7]:
dataframe['duration_sec'].std()

3149.681666336765

## In Minutes divide the above result with 60 

In [8]:
dataframe['duration_sec'].std()/60

52.494694438946084

the other way to get average and standard deviation is by using .describe()

## Finding the busiest dates:

####  Upon on carefully reviewing the ford go bike plan details at "https://www.fordgobike.com/pricing" a trip can last upto a month

In [9]:
# first lets make a dummy dataframe of just the start time and the end time and call it dummy

dummy  = dataframe[['start_time', 'end_time']]

dummy.head()

,start_time,end_time
0,2017-12-31 16:57:39.6540,2018-01-01 15:12:50.2450
1,2017-12-31 15:56:34.8420,2018-01-01 13:49:55.6170
2,2017-12-31 22:45:48.4110,2018-01-01 11:28:36.8830
3,2017-12-31 17:31:10.6360,2018-01-01 10:47:23.5310
4,2017-12-31 14:23:14.0010,2018-01-01 02:29:57.5710


In [10]:
# the dummy dataframe consists of data and time. for us to analyze the busy dates we need to get rid of the time 
# lets use python split function to seperate data and time 

#let us write a helper function to do our task and call it when necessary

def date(string):
    return (string.split(' ')[0])

In [11]:
dummy['start_time'] = dummy['start_time'].apply(date)
dummy['end_time'] = dummy['end_time'].apply(date)

In [12]:
dummy.head()

,start_time,end_time
0,2017-12-31,2018-01-01
1,2017-12-31,2018-01-01
2,2017-12-31,2018-01-01
3,2017-12-31,2018-01-01
4,2017-12-31,2018-01-01


In [13]:
count = dummy.start_time.value_counts()
# count.head()
# from here we can say that the busy day is march 29 2018 but we cannot come to the conclusion taking jus the starttime
# what if the bike is rented for a month or 3 days using monthly or 3 day pass.
#here we need to consider the mutiple rental records.
#for this the startdate and end date varies and start date is always less than the end date.

multiple = dummy.loc[(dummy['start_time']< dummy['end_time'])]
#gives only the values for which multiple dates is true.

In [14]:
# now let us loop through the multiple data frame and append the number of days between the start and end to the
# date in the count dataframe.

from datetime import *

for index, row in multiple.iterrows():
    start = datetime.strptime(row['start_time'], "%Y-%m-%d")
    end = datetime.strptime(row['end_time'], "%Y-%m-%d")
    delta = end - start
    for i in range(1, delta.days+1):
        date = str((start+timedelta(i)).date())
        if date in count:
            count[date] +=1
        else:
            count[date] = 1   

In [15]:
count.head()

2018-03-29    5445
2018-03-28    5372
2018-02-08    5219
2018-03-27    5192
2018-02-06    5167
Name: start_time, dtype: int64

## The busiest day is on march 29 2018